In [5]:
import pandas as pd
df = pd.read_csv("news_data_extra.csv")
print(df.columns)
print(df.head(3))

Index(['title', 'subtitle', 'author', 'url', 'text', 'time', 'date',
       'difficulty', 'tags'],
      dtype='object')
                                               title subtitle          author  \
0  Вакуумные флуктуации нарушили механизм квантов...      NaN  Марат Хамадеев   
1  Начались наблюдения на самом чувствительном де...      NaN  Тимур Кешелава   
2  Атомный ледокол «Арктика» вышел на ходовые исп...      NaN   Василий Сычев   

                                                url  \
0  https://nplus1.ru/news/2022/03/03/vacuum-to-hall   
1          https://nplus1.ru/news/2019/04/26/LHAASO   
2          https://nplus1.ru/news/2019/12/12/arctic   

                                                text   time      date  \
0  Европейские физики экспериментально доказали, ...  21:00  03.03.22   
1  В Китае стартовала\nнаучная программа новой ус...  15:16  26.04.19   
2  Перспективный российский атомный ледокол «Аркт...  14:36  12.12.19   

   difficulty                      tags 

In [6]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
index_name = "news_extra_extra3"


In [7]:
mapping = {
    "settings": {
        "analysis": {
            "filter": {
                "russian_stop": {
                    "type": "stop",
                    "stopwords": "_russian_"  # стандартные русские стоп-слова
                },
                "russian_stemmer": {
                    "type": "stemmer",
                    "language": "russian"
                }
            },
            "analyzer": {
                "russian_custom": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "russian_stop",
                        "russian_stemmer"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {"type": "text", "analyzer": "russian_custom"},
            "subtitle": {"type": "text", "analyzer": "russian_custom"},
            "author": {"type": "keyword"},
            "url": {"type": "keyword", "index": False},
            "text": {"type": "text", "analyzer": "russian_custom"},
            "time": {"type": "keyword"},
            "date": {"type": "date", "format": "dd.MM.yy||yyyy-MM-dd"},
            "difficulty": {"type": "float"},
            "tags": {"type": "keyword"},
            "true_title": {"type": "text", "index": False},
            "content_vector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body=mapping)
print(f"Индекс {index_name} успешно создан с русским анализатором")

Индекс news_extra_extra3 успешно создан с русским анализатором


In [8]:
import csv
from normalize_utils import (
    normalize_date,
    normalize_difficulty,
    normalize_tags,
    lemmatize_text,
    clean_text
)
from synonyms_utils import get_sentence_embedding

with open("news_data_extra.csv", newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        row['true_title'] = row['title']
        row["date"] = normalize_date(row.get("date", ""))
        row["difficulty"] = normalize_difficulty(row.get("difficulty"))
        row["tags"] = normalize_tags(row.get("tags"))

        # Лемматизируем большие текстовые поля
        for field in ["title", "subtitle", "text"]:
            if row.get(field):
                row[field] = lemmatize_text(clean_text(row[field]))

        title = " ".join(row["title"]) if isinstance(row.get("title"), list) else row.get("title", "")
        subtitle = " ".join(row["subtitle"]) if isinstance(row.get("subtitle"), list) else row.get("subtitle", "")

        content_text = ". ".join(filter(None, [title, subtitle]))

        row["content_vector"] = get_sentence_embedding(content_text)[0].tolist()
        es.index(index=index_name, document=row)

print("CSV-файл успешно предобработан и загружен в Elasticsearch")


c:\Users\vadim\Desktop\7 сем\Поиск\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CSV-файл успешно предобработан и загружен в Elasticsearch
